***Ratios***
- Standalone -> https://www.moneycontrol.com/financials/itc/ratiosVI/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/consolidated-ratiosVI/ITC#ITC

***Balance Sheet***
- Standalone -> https://www.moneycontrol.com/financials/itc/balance-sheetVI/ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/consolidated-balance-sheetVI/ITC
- for next page https://www.moneycontrol.com/financials/itc/profit-lossVI/ITC/2#ITC

***Profit & Loss***- 
- Standalone -> https://www.moneycontrol.com/financials/itc/profit-lossVI/ITC
- Consolidated ->https://www.moneycontrol.com/financials/itc/consolidated-balance-sheetVI/ITC#ITC

***Quarterly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/quarterly-results/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-quarterly-results/ITC#ITC

***Half Yearly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/half-yearly/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-half-yearly/ITC#ITC

***Nine Months Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/nine-months/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-nine-months/ITC#ITC

***Yearly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/yearly/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-yearly/ITC#ITC

***Cash Flow***
- Standalone - https://www.moneycontrol.com/financials/itc/cash-flowVI/ITC#ITC
- Consolidated - https://www.moneycontrol.com/financials/itc/consolidated-cash-flowVI/ITC#ITC

***Capital Structure***
- https://www.moneycontrol.com/financials/itc/capital-structure/ITC#ITC

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd

In [5]:
mc_fin_baseurl = 'https://www.moneycontrol.com/financials'

***To download Financial ratios***

In [10]:
def processfindf(df):
    df.at[0,0] = 'item'
    df = df.iloc[:, :-1]
    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0])
    df.set_index('item', inplace=True)
    return df

# https://www.moneycontrol.com/financials/itc/consolidated-balance-sheetVI/ITC

# fintp -> 'ratios', 'balance-sheet', 'profit-loss', 'yearly', 'quarterly-results', 'half-yearly', 'nine-months'
def getfinancials(symbol, fintp='ratios'):
    symbol = symbol.upper()
    url = f'{mc_fin_baseurl}/{symbol}/consolidated-{fintp}VI/{symbol}'
    if fintp in ['yearly', 'quarterly-results', 'half-yearly', 'nine-months']:
        url = f'{mc_fin_baseurl}/{symbol}/results/consolidated-{fintp}/{symbol}'
    lst = [pd.read_html(f'{url}/{i}') for i in range(2, 6)]
    lst = [pd.read_html(url)] + lst
    dfs = [processfindf(i[0]) for i in lst if i]
    df = pd.concat(dfs, axis=1)
    df = df.dropna(how='all') 
    df.fillna(0, inplace=True)
    if fintp in ['balance-sheet', 'profit-loss']: df = df.iloc[1:]
    return df

In [ ]:
%%time
# df = getfinancials('HDF01', 'ratios')
# df = getfinancials('HDF01', 'balance-sheet')
# df = getfinancials('HDF01', 'profit-loss')
# df = getfinancials('HDF01', 'yearly')
# df = getfinancials('HDF01', 'half-yearly')
# df = getfinancials('HDF01', 'quarterly-results')
# df = getfinancials('HDF01', 'nine-months')

In [11]:
df = getfinancials('sbi', 'profit-loss')

In [12]:
df

,Mar 20,Mar 19,Mar 18,Mar 17,Mar 16,Mar 20,Mar 19,Mar 18,Mar 17,Mar 16,...,Mar 20,Mar 19,Mar 18,Mar 17,Mar 16,Mar 20,Mar 19,Mar 18,Mar 17,Mar 16
item,,,,,,,,,,,,,,,,,,,,,
Interest / Discount on Advances / Bills,185494.20,166124.58,144958.59,156790.48,157001.75,185494.20,166124.58,144958.59,156790.48,157001.75,...,185494.20,166124.58,144958.59,156790.48,157001.75,185494.20,166124.58,144958.59,156790.48,157001.75
Income from Investments,74812.87,80243.51,75036.62,64201.37,57922.72,74812.87,80243.51,75036.62,64201.37,57922.72,...,74812.87,80243.51,75036.62,64201.37,57922.72,74812.87,80243.51,75036.62,64201.37,57922.72
Interest on Balance with RBI and Other Inter-Bank funds,3066.25,1324.76,2410.75,2591.57,1186.80,3066.25,1324.76,2410.75,2591.57,1186.80,...,3066.25,1324.76,2410.75,2591.57,1186.80,3066.25,1324.76,2410.75,2591.57,1186.80
Others,6478.34,5629.29,6564.32,6864.07,5743.57,6478.34,5629.29,6564.32,6864.07,5743.57,...,6478.34,5629.29,6564.32,6864.07,5743.57,6478.34,5629.29,6564.32,6864.07,5743.57
Total Interest Earned,269851.66,253322.14,228970.28,230447.49,221854.84,269851.66,253322.14,228970.28,230447.49,221854.84,...,269851.66,253322.14,228970.28,230447.49,221854.84,269851.66,253322.14,228970.28,230447.49,221854.84
Other Income,92377.43,76898.74,77557.24,68192.96,51016.18,92377.43,76898.74,77557.24,68192.96,51016.18,...,92377.43,76898.74,77557.24,68192.96,51016.18,92377.43,76898.74,77557.24,68192.96,51016.18
Total Income,362229.09,330220.88,306527.52,298640.45,272871.03,362229.09,330220.88,306527.52,298640.45,272871.03,...,362229.09,330220.88,306527.52,298640.45,272871.03,362229.09,330220.88,306527.52,298640.45,272871.03
Interest Expended,161123.80,155867.46,146602.98,149114.67,143047.36,161123.80,155867.46,146602.98,149114.67,143047.36,...,161123.80,155867.46,146602.98,149114.67,143047.36,161123.80,155867.46,146602.98,149114.67,143047.36
Payments to and Provisions for Employees,48850.95,43795.01,35410.62,35691.21,32525.60,48850.95,43795.01,35410.62,35691.21,32525.60,...,48850.95,43795.01,35410.62,35691.21,32525.60,48850.95,43795.01,35410.62,35691.21,32525.60
